<a href="https://www.kaggle.com/code/pogscafe/comfyui-stable-video-diffusion?scriptVersionId=226725174" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Installation
The notebook requres a GPU Accelerator to run. To keep the files saved for the next run, make sure Persistence is set to "Files only" or "Variables and Files".  
For examples, have a look at our [SVD quickstart guide](https://www.pogs.cafe/content/svd-comfyui-kaggle).

### If the video preview doesn't show up after generation, use File Browser to download the video. The output folder is:  /kaggle/working/ComfyUI/output

In [ ]:
# --- Parameters --- #

# If set to false, ComfyUI and ComfyUI Manager will not be updated after the initial install.
# Set to true to update.
update_comfy = False
update_manager = False
install_sdxl_model = True

# SDXL model that will be saved to permanent storage. Used for text-to-image.
sdxl_model_url = 'https://civitai.com/api/download/models/238308?type=Model&format=SafeTensor&size=pruned&fp=fp16'
sdxl_model_name = 'AlbedoBase.safetensors'

# Stable video diffusion model that will be saved to permanent storage.
# Used for image-to-video.
video_model_url_s = 'https://huggingface.co/becausecurious/stable-video-diffusion-img2vid-fp16/resolve/main/svd-fp16.safetensors?download=true'
video_model_name_s = 'svd-fp16.safetensors'
video_model_url = 'https://huggingface.co/becausecurious/stable-video-diffusion-img2vid-fp16/resolve/main/svd_xt-fp16.safetensors?download=true'
video_model_name = 'svd_xt-fp16.safetensors'

# ------------------- #

from os import path

%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
!git checkout 66831eb6e96cd974fb2d0fc4f299b23c6af16685

if update_comfy:
    get_ipython().system('git pull')
!pip install -r requirements.txt

working_folder = '/kaggle/working'
temp_folder = '/kaggle/temp'
checkpoints =  f'{working_folder}/ComfyUI/models/checkpoints'
loras = f'{working_folder}/ComfyUI/models/loras'
temp_loras_link = loras + '/temp-loras'
temp_loras = f'{temp_folder}/temp-loras'
temp_models_link = checkpoints + '/temp-models'
temp_models = f'{temp_folder}/temp-models'
custom_nodes = f'{working_folder}/ComfyUI/custom_nodes'
outputs = f'{working_folder}/ComfyUI/output'
zip_outputs = f'{working_folder}/outputs'

!mkdir $temp_folder
!mkdir $temp_models
!mkdir $temp_loras

if not path.exists(temp_models_link):
    get_ipython().system(f'ln -s {temp_models} {checkpoints}')
if not path.exists(temp_loras_link):
    get_ipython().system(f'ln -s {temp_loras} {loras}')
    
!mamba install openssh -y

# Install the node manager
%cd $working_folder/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
if update_manager:
    get_ipython().system('git pull')
!pip install -r requirements.txt

# Install initial models
%cd $checkpoints
get_ipython().system(f'wget -O "{video_model_name}" "{video_model_url}"')
get_ipython().system(f'wget -O "{video_model_name_s}" "{video_model_url_s}"')
if install_sdxl_model and not path.exists(f'{checkpoints}/{sdxl_model_name}'):
    get_ipython().system(f'wget -O "{sdxl_model_name}" "{sdxl_model_url}"')

# Install an initial LoRA
%cd $loras
model_url = 'https://civitai.com/api/download/models/137124?type=Model&format=SafeTensor'
model_name = 'DreamArt.safetensors'
if not path.exists(f'{loras}/{model_name}'):
    get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# Install the logic nodes
%cd $custom_nodes
!git clone https://github.com/theUpsider/ComfyUI-Logic.git
    
# Install VideoHelperSuite nodes
%cd $custom_nodes
!rm -rf ComfyUI-VideoHelperSuite
get_ipython().system(f'wget -O "videohelpersuite.zip" "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite/archive/4de52b66c809fb48b69d03019935da7a64ac40a4.zip"')
import shutil
shutil.unpack_archive("videohelpersuite.zip", ".", "zip") 
!mv ComfyUI-VideoHelperSuite* ComfyUI-VideoHelperSuite
%cd ComfyUI-VideoHelperSuite

# Frame interpolation nodes
%cd $custom_nodes
!git clone https://github.com/Fannovel16/ComfyUI-Frame-Interpolation
%cd ComfyUI-Frame-Interpolation
!git pull
!python install.py

# Modded nodes, used for math in the text-to-video and image-to-video workflow
%cd $custom_nodes
!git clone https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes
%cd Derfuu_ComfyUI_ModdedNodes
!git checkout 6af9d06632e30b9b00eced4811cb61f52171549e

!pip install --force-reinstall torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --force-reinstall numpy==1.22

# 
# Start the WebUI

Workflows can be found on https://github.com/wandaweb/ComfyUI-Kaggle/tree/main/sample-workflows

**Option 1: Starting the Web UI with Pinggy**  
* Wait for the UI to start.  
* Click the link that ends with .pinggy.link 😁
* If generation is still running after the link expires in an hour, wait for the generation to complete and restart the WebUI code block to get a new link
* If the generation completed successfully, but the video didn't show up in the preview, you can find it in the output folder

In [2]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python {working_folder}/ComfyUI/main.py & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()


waiting for output
[START] Security scan
waiting for output
Allocated port 7 for remote forward to localhost:8188
[DONE] Security scan
Failed to execute startup-script: /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Manager/prestartup_script.py / module 'folder_paths' has no attribute 'get_user_directory'

Prestartup times for custom nodes:
   2.6 seconds (PRESTARTUP FAILED): /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Manager

😁 😁 😁
URL: http://rnyni-34-55-81-164.a.free.pinggy.link
😁 😁 😁
7=)0]8;;\                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

KeyboardInterrupt: 

***
**Option 2: Starting the Web UI with ngrok**  
* Make sure to put your ngrok token in the Ngrok_token variable. The token can be obtained from https://ngrok.com
* If you have a static domain, put your ngrok domain in the Ngrok_domain variable.
* Wait for the line that says "To see the GUI go to: http://127.0.0.1:8188 " 
* Visit your ngrok URL (either your static domain, or the ngrok url displayed in the output)

In [ ]:
# - Option 2: Running with Ngrok - #

Ngrok_token = "" #@param {type:"string"}
# Put your ngrok token here (obtainable from https://ngrok.com)

Ngrok_domain = "" # optional, leave empty if you don't have a domain

# -------------------------------- #

!pip install pyngrok

from pyngrok import ngrok, conf
import fileinput
import sys

if Ngrok_token!="":
  ngrok.kill()
  srv=ngrok.connect(8188 , pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token),
                    bind_tls=True, domain=Ngrok_domain).public_url
  print(srv)
  get_ipython().system(f"python {working_folder}/ComfyUI/main.py")
else:
  print('An ngrok token is required. You can get one on https://ngrok.com and paste it into the ngrok_token field.')

# Install a LoRA

Copy the model URL to the model_url field. Make sure the model can be accessed publicly, without being signed into a website.

In [ ]:
# Install a LoRA in temporary storage
model_url = 'https://huggingface.co/HarroweD/HarrlogosXL/resolve/main/Harrlogos_v2.0.safetensors?download=true'
model_name = 'Harrlogos_v2.0.safetensors'

# Other model examples
#model_url = 'https://civitai.com/api/download/models/153734?type=Model&format=SafeTensor'
#model_name = 'DarkFantasy.safetensors'

#model_url = 'https://civitai.com/api/download/models/81907'
#model_name = 'CyberpunkAI.safetensors'

#model_url = 'https://civitai.com/api/download/models/217866?type=Model&format=SafeTensor'
#model_name = 'Wowifier.safetensors'

%cd $temp_loras
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a LoRA in permanent storage
model_url = 'https://civitai.com/api/download/models/132727'
model_name = 'Samaritan.safetensors'

%cd $loras
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# 
# Install a Model
Copy the model URL to the model_url field. Make sure the model can be accessed publicly, without being signed into a website.

In [ ]:
# Install a model in temporary storage
model_url = 'https://civitai.com/api/download/models/251662'
model_name = 'DreamShaperXL-Turbo.safetensors'

%cd $temp_models
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

In [ ]:
# Install a model in permanent storage
# Please check there is enough space and, if needed, delete a previous model using the "Delete a model" block.
model_url = 'https://civitai.com/api/download/models/198962?type=Model&format=SafeTensor&size=pruned&fp=fp16'
model_name = 'DynaVision.safetensors'

%cd $checkpoints
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# 
# Download a model for a custom node

In [ ]:
model_folder = f'{working_folder}/ComfyUI/custom_nodes/my_node/models'
model_url = ''
model_name = 'model.safetensors'

%cd $model_folder
get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

# 
# List permanent models

In [ ]:
# List permanent models
!ls -la $checkpoints

# 
# Check a model's size

In [ ]:
# Check the size of a model
model_to_check = '/kaggle/working/ComfyUI/models/checkpoints/svd-fp16.safetensors'
!du -sh $model_to_check 

# 
# Find files larger than 100M

In [ ]:
!find $working_folder -size +100M

# 
# Delete a file

In [ ]:
# Delete a file to free up space
model_to_delete = '/kaggle/working/ComfyUI/models/checkpoints/model.safetensors'
!rm $model_to_delete

# 
# Move outputs to a .zip file

In [ ]:
# Create a .zip file 
import datetime
import shutil 
import os.path
!mkdir $working_folder/outputs

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
zipfile = shutil.make_archive(f'{working_folder}/outputs/output_{timestamp}', 'zip', f'{working_folder}/ComfyUI/output')

if os.path.exists(f'{working_folder}/outputs/output_{timestamp}.zip'):
   print(f'Created {zipfile}') 
else: 
   print("ZIP file not created")

In [ ]:
# Delete output files. 
# To avoid losing work, please make sure the ZIP file contains your outputs before running this step.

!rm $outputs/*.*

---
# File Browser

### Install and configure FileBrowser


In [ ]:
%cd /kaggle
!wget https://github.com/filebrowser/filebrowser/releases/download/v2.27.0/linux-amd64-filebrowser.tar.gz
!tar xvfz linux-amd64-filebrowser.tar.gz
!chmod a+x /kaggle/filebrowser
!rm /kaggle/config.json
!/kaggle/filebrowser config init 
!/kaggle/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/filebrowser config set --branding.theme=dark > /dev/null
!/kaggle/filebrowser users add admin admin 
!/kaggle/filebrowser config export "/kaggle/config.json"
#!cat /kaggle/config.json # show the config file

### Start FileBrowser

If the app greets you with a login screen, the username and password are both "admin", which was set in the configuration code block above.
Click the link ending with pinggy.link to open the File Browser.

In [ ]:
# Starting FileBrowser with Pinggy

%cd /kaggle
        
from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    !/kaggle/filebrowser -c "/kaggle/config.json" & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8080 a.pinggy.io > log.txt > log.txt
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

# 
# Delete the working folder

In [ ]:
!rm -rf /kaggle/working/*